# table-1

In [13]:
##############################################################################################
#  512 x 384 mf3 mf5 avg gau res jpeg ###############################################################################################


import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd

# Load features with labels (last column = label)
def load_data_with_labels(file_path):
    data = np.load(file_path)
    X = data[:, :-1]
    y = data[:, -1]
    return X, y

# Load all datasets
X_orig, y_orig = load_data_with_labels('512x384_orig_unc.npy')
X_avg, y_avg = load_data_with_labels('512x384_avg_unc.npy')
X_gau, y_gau = load_data_with_labels('512x384_gau_unc.npy')
X_jpeg, y_jpeg = load_data_with_labels('512x384_jpeg_90.npy')
X_res, y_res = load_data_with_labels('512x384_res_unc.npy')
X_mf3, y_mf3 = load_data_with_labels('512x384_mf3_unc.npy')
X_mf5, y_mf5 = load_data_with_labels('512x384_mf5_unc.npy')

# Split based on original dataset
indices = np.arange(len(y_orig))
train_indices, test_indices = train_test_split(indices, test_size=0.25, random_state=42, stratify=y_orig)
np.save('test_indices_1.npy', test_indices)

# Prepare training data using consistent indices
X_train = np.vstack([
    X_orig[train_indices], X_avg[train_indices], X_gau[train_indices],
    X_jpeg[train_indices], X_res[train_indices], X_mf3[train_indices], X_mf5[train_indices]
])
y_train = np.hstack([
    y_orig[train_indices], y_avg[train_indices], y_gau[train_indices],
    y_jpeg[train_indices], y_res[train_indices], y_mf3[train_indices], y_mf5[train_indices]
])

# Train SVM model
model_1 = SVC(kernel='poly', degree=3, C=1.0, probability=True)
model_1.fit(X_train, y_train)

# Define comparison pairs
pairs = [
    ('mf3', 'avg'), ('mf3', 'gau'), ('mf3', 'jpeg'), ('mf3', 'res'),
    ('mf5', 'avg'), ('mf5', 'gau'), ('mf5', 'jpeg'), ('mf5', 'res')
]

# Dataset dictionary
data_dict = {
    'avg': (X_avg, y_avg),
    'gau': (X_gau, y_gau),
    'jpeg': (X_jpeg, y_jpeg),
    'res': (X_res, y_res),
    'mf3': (X_mf3, y_mf3),
    'mf5': (X_mf5, y_mf5)
}

# Prepare evaluation
poe_row = []
auc_row = []

# Evaluate each pair
for a, b in pairs:
    X_a, y_a = data_dict[a]
    X_b, y_b = data_dict[b]

    # Select test samples using original indices
    Xa_test = X_a[test_indices]
    Xb_test = X_b[test_indices]

    ya_test = np.ones(len(test_indices))  # label 1 for attack a
    yb_test = np.zeros(len(test_indices))  # label 0 for attack b

    # Combine for evaluation
    X_pair = np.vstack([Xa_test, Xb_test])
    y_pair = np.hstack([ya_test, yb_test])

    y_pred = model_1.predict(X_pair)
    y_proba = model_1.predict_proba(X_pair)[:, 1]

    poe = 1 - accuracy_score(y_pair, y_pred)
    auc = roc_auc_score(y_pair, y_proba)

    poe_row.append(poe)
    auc_row.append(auc)

# Create result table
table = pd.DataFrame([poe_row, auc_row],
                     index=['PoE', 'AUC'],
                     columns=[f'{a} vs {b}' for a, b in pairs])

print("Evaluation Table:")
print(table.to_string(float_format="%.4f"))


Evaluation Table:
     mf3 vs avg  mf3 vs gau  mf3 vs jpeg  mf3 vs res  mf5 vs avg  mf5 vs gau  mf5 vs jpeg  mf5 vs res
PoE      0.0075      0.0075       0.0090      0.0090      0.0060      0.0060       0.0075      0.0075
AUC      0.9952      0.9992       0.9950      0.9958      0.9949      0.9976       0.9941      0.9953


In [14]:
##############################################################################################
#  256 x 256 mf3 mf5 avg gau res jpeg ###############################################################################################

import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score
import pandas as pd

# Load features with labels
def load_data_with_labels(file_path):
    data = np.load(file_path)
    X = data[:, :-1]
    y = data[:, -1]
    return X, y

# Load 256x256 datasets
X_orig_256, y_orig_256 = load_data_with_labels('256x256_orig_unc.npy')
X_avg_256, y_avg_256 = load_data_with_labels('256x256_avg_unc.npy')
X_gau_256, y_gau_256 = load_data_with_labels('256x256_gau_unc.npy')
X_jpeg_256, y_jpeg_256 = load_data_with_labels('256x256_jpeg_90.npy')
X_res_256, y_res_256 = load_data_with_labels('256x256_res_unc.npy')
X_mf3_256, y_mf3_256 = load_data_with_labels('256x256_mf3_unc.npy')
X_mf5_256, y_mf5_256 = load_data_with_labels('256x256_mf5_unc.npy')

# Use previously defined `test_indices` (from 512x384_orig_unc)
# If running this separately, you need to re-import or re-calculate them
# For context:
# indices = np.arange(len(y_orig_512))
# train_indices, test_indices = train_test_split(indices, test_size=0.25, random_state=42, stratify=y_orig_512)

# Replace with appropriate length if you don't have it already
# For example purposes, using this (but use your actual test_indices!):
test_indices = np.load('test_indices_1.npy')  # load from file if previously saved

# Define dataset dictionary for 256x256
data_dict_256 = {
    'avg': (X_avg_256, y_avg_256),
    'gau': (X_gau_256, y_gau_256),
    'jpeg': (X_jpeg_256, y_jpeg_256),
    'res': (X_res_256, y_res_256),
    'mf3': (X_mf3_256, y_mf3_256),
    'mf5': (X_mf5_256, y_mf5_256)
}

# Define the same comparison pairs
pairs = [
    ('mf3', 'avg'), ('mf3', 'gau'), ('mf3', 'jpeg'), ('mf3', 'res'),
    ('mf5', 'avg'), ('mf5', 'gau'), ('mf5', 'jpeg'), ('mf5', 'res')
]

# Evaluate using the model previously trained on 512x384
poe_row = []
auc_row = []

for a, b in pairs:
    X_a, y_a = data_dict_256[a]
    X_b, y_b = data_dict_256[b]

    Xa_test = X_a[test_indices]
    Xb_test = X_b[test_indices]

    ya_test = np.ones(len(test_indices))  # attack A
    yb_test = np.zeros(len(test_indices))  # attack B

    X_pair = np.vstack([Xa_test, Xb_test])
    y_pair = np.hstack([ya_test, yb_test])

    y_pred = model_1.predict(X_pair)
    y_proba = model_1.predict_proba(X_pair)[:, 1]

    poe = 1 - accuracy_score(y_pair, y_pred)
    auc = roc_auc_score(y_pair, y_proba)

    poe_row.append(poe)
    auc_row.append(auc)

# Generate evaluation table
table = pd.DataFrame([poe_row, auc_row],
                     index=['PoE', 'AUC'],
                     columns=[f'{a} vs {b}' for a, b in pairs])

print("Evaluation Table on 256x256 data:")
print(table.to_string(float_format="%.4f"))


Evaluation Table on 256x256 data:
     mf3 vs avg  mf3 vs gau  mf3 vs jpeg  mf3 vs res  mf5 vs avg  mf5 vs gau  mf5 vs jpeg  mf5 vs res
PoE      0.4343      0.4388       0.4239      0.4493      0.4358      0.4403       0.4254      0.4507
AUC      0.7454      0.7162       0.7511      0.6087      0.8037      0.7807       0.8181      0.6822


In [15]:
###################################################################################################
# 512 x 384 mf35 all
#####################################################################################################


import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd

# Load features with labels (last column = label)
def load_data_with_labels(file_path):
    data = np.load(file_path)
    X = data[:, :-1]
    y = data[:, -1]
    return X, y

# Load datasets
X_all, y_all = load_data_with_labels('512x384_all_unc.npy')
X_mf35, y_mf35 = load_data_with_labels('512x384_mf35_unc.npy')

# Split data (75% train, 25% test) using consistent indices
indices = np.arange(len(y_all))
train_indices, test_indices = train_test_split(indices, test_size=0.25, random_state=42, stratify=y_all)
np.save('test_indices_2.npy', test_indices)

# Prepare training data
X_train = np.vstack([X_all[train_indices], X_mf35[train_indices]])
y_train = np.hstack([y_all[train_indices], y_mf35[train_indices]])

# Train SVM model
model_2 = SVC(kernel='poly', degree=3, C=1.0, probability=True)
model_2.fit(X_train, y_train)

# Create dataset dictionary for evaluation
data_dict = {
    'all': (X_all, y_all),
    'mf35': (X_mf35, y_mf35)
}

# Define comparison pairs for evaluation
pairs = [
    ('mf35', 'all')
]

# Prepare evaluation
poe_row = []
auc_row = []

# Evaluate each pair
for a, b in pairs:
    X_a, y_a = data_dict[a]
    X_b, y_b = data_dict[b]

    Xa_test = X_a[test_indices]
    Xb_test = X_b[test_indices]

    ya_test = np.ones(len(test_indices))  # label 1 for attack a
    yb_test = np.zeros(len(test_indices))  # label 0 for attack b

    # Combine for evaluation
    X_pair = np.vstack([Xa_test, Xb_test])
    y_pair = np.hstack([ya_test, yb_test])

    y_pred = model_2.predict(X_pair)
    y_proba = model_2.predict_proba(X_pair)[:, 1]

    poe = 1 - accuracy_score(y_pair, y_pred)
    auc = roc_auc_score(y_pair, y_proba)

    poe_row.append(poe)
    auc_row.append(auc)

# Create result table
table = pd.DataFrame([poe_row, auc_row],
                     index=['PoE', 'AUC'],
                     columns=[f'{a} vs {b}' for a, b in pairs])

print("Evaluation Table:")
print(table.to_string(float_format="%.4f"))


Evaluation Table:
     mf35 vs all
PoE       0.0075
AUC       0.9942


In [16]:
###################################################################################################
# 256 x 256 mf35 all
#####################################################################################################

import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score
import pandas as pd
import joblib  # optional if you decide to save/load model

# Define function again (in case you're running standalone)
def load_data_with_labels(file_path):
    data = np.load(file_path)
    X = data[:, :-1]
    y = data[:, -1]
    return X, y

# Load previously used test indices
# X_all_512, y_all_512 = load_data_with_labels('512x384_all_unc.npy')
# indices = np.arange(len(y_all_512))
# _, test_indices = train_test_split(indices, test_size=0.25, random_state=42, stratify=y_all_512)

test_indices = np.load('test_indices_2.npy')

# Load trained model (if saved to file, otherwise assume `model` is in memory)
# model = joblib.load('trained_model_512_poly.pkl')

# Load 256x256 datasets
X_all_256, y_all_256 = load_data_with_labels('256x256_all_unc.npy')
X_mf35_256, y_mf35_256 = load_data_with_labels('256x256_mf35_unc.npy')

# Create dataset dictionary
data_dict = {
    '256_all': (X_all_256, y_all_256),
    '256_mf35': (X_mf35_256, y_mf35_256)
}

# Define comparison pair
pairs = [('256_mf35', '256_all')]

# Evaluate
poe_row = []
auc_row = []

for a, b in pairs:
    X_a, y_a = data_dict[a]
    X_b, y_b = data_dict[b]

    Xa_test = X_a[test_indices]
    Xb_test = X_b[test_indices]

    ya_test = np.ones(len(test_indices))  # label 1 for attack a
    yb_test = np.zeros(len(test_indices))  # label 0 for attack b

    X_pair = np.vstack([Xa_test, Xb_test])
    y_pair = np.hstack([ya_test, yb_test])

    y_pred = model_2.predict(X_pair)
    y_proba = model_2.predict_proba(X_pair)[:, 1]

    poe = 1 - accuracy_score(y_pair, y_pred)
    auc = roc_auc_score(y_pair, y_proba)

    poe_row.append(poe)
    auc_row.append(auc)

# Create result table
table = pd.DataFrame([poe_row, auc_row],
                     index=['PoE', 'AUC'],
                     columns=[f'{a} vs {b}' for a, b in pairs])

print("Evaluation Table (on 256x256 data):")
print(table.to_string(float_format="%.4f"))


 

Evaluation Table (on 256x256 data):
     256_mf35 vs 256_all
PoE               0.4222
AUC               0.7836


# table-2

In [17]:
#############################################################################################################
# 512 x 384 ori mf3q mf5q
#############################################################################################################


import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score
import pandas as pd

# Load features with labels
def load_data_with_labels(file_path):
    data = np.load(file_path)
    X = data[:, :-1]
    y = data[:, -1]
    return X, y

# Load specified 512x384 datasets
X_orig, y_orig = load_data_with_labels('512x384_orig_unc.npy')
# X_mf3, y_mf3 = load_data_with_labels('512x384_mf3_unc.npy')
# X_mf5, y_mf5 = load_data_with_labels('512x384_mf5_unc.npy')
X_mf3_90, y_mf3_90 = load_data_with_labels('512x384_mf3_90.npy')
X_mf5_90, y_mf5_90 = load_data_with_labels('512x384_mf5_90.npy')

# Load saved test indices
test_indices = np.load('test_indices_1.npy')

# Define dataset dictionary
data_dict = {
    'orig': (X_orig, y_orig),
    # 'mf3': (X_mf3, y_mf3),
    # 'mf5': (X_mf5, y_mf5),
    'mf3_90': (X_mf3_90, y_mf3_90),
    'mf5_90': (X_mf5_90, y_mf5_90)
}

# Define comparison pairs
pairs = [
    ('mf3_90', 'orig'), ('mf5_90', 'orig')
]

# Evaluate using the previously trained SVM model (ensure `model_1` exists in memory)
poe_row = []
auc_row = []

for a, b in pairs:
    X_a, y_a = data_dict[a]
    X_b, y_b = data_dict[b]

    Xa_test = X_a[test_indices]
    Xb_test = X_b[test_indices]

    ya_test = np.ones(len(test_indices))  # label for attack
    yb_test = np.zeros(len(test_indices))  # label for original

    X_pair = np.vstack([Xa_test, Xb_test])
    y_pair = np.hstack([ya_test, yb_test])

    y_pred = model_1.predict(X_pair)
    y_proba = model_1.predict_proba(X_pair)[:, 1]

    poe = 1 - accuracy_score(y_pair, y_pred)
    auc = roc_auc_score(y_pair, y_proba)

    poe_row.append(poe)
    auc_row.append(auc)

# Generate and print results table
table = pd.DataFrame([poe_row, auc_row],
                     index=['PoE', 'AUC'],
                     columns=[f'{a} vs {b}' for a, b in pairs])

print("Evaluation Table on 512x384 - attack vs original:")
print(table.to_string(float_format="%.4f"))



Evaluation Table on 512x384 - attack vs original:
     mf3_90 vs orig  mf5_90 vs orig
PoE          0.2791          0.0657
AUC          0.9858          0.9885


In [18]:
#############################################################################################################
# 256 x 256 ori mf3q mf5q
#############################################################################################################


import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score
import pandas as pd

# Load features with labels
def load_data_with_labels(file_path):
    data = np.load(file_path)
    X = data[:, :-1]
    y = data[:, -1]
    return X, y

# Load specified 512x384 datasets
X_orig, y_orig = load_data_with_labels('256x256_orig_unc.npy')
# X_mf3, y_mf3 = load_data_with_labels('256x256_mf3_unc.npy')
# X_mf5, y_mf5 = load_data_with_labels('256x256_mf5_unc.npy')
X_mf3_90, y_mf3_90 = load_data_with_labels('256x256_mf3_90.npy')
X_mf5_90, y_mf5_90 = load_data_with_labels('256x256_mf5_90.npy')

# Load saved test indices
test_indices = np.load('test_indices_1.npy')

# Define dataset dictionary
data_dict = {
    'orig': (X_orig, y_orig),
    # 'mf3': (X_mf3, y_mf3),
    # 'mf5': (X_mf5, y_mf5),
    'mf3_90': (X_mf3_90, y_mf3_90),
    'mf5_90': (X_mf5_90, y_mf5_90)
}

# Define comparison pairs
pairs = [
    ('mf3_90', 'orig'), ('mf5_90', 'orig')
]

# Evaluate using the previously trained SVM model (ensure `model_1` exists in memory)
poe_row = []
auc_row = []

for a, b in pairs:
    X_a, y_a = data_dict[a]
    X_b, y_b = data_dict[b]

    Xa_test = X_a[test_indices]
    Xb_test = X_b[test_indices]

    ya_test = np.ones(len(test_indices))  # label for attack
    yb_test = np.zeros(len(test_indices))  # label for original

    X_pair = np.vstack([Xa_test, Xb_test])
    y_pair = np.hstack([ya_test, yb_test])

    y_pred = model_1.predict(X_pair)
    y_proba = model_1.predict_proba(X_pair)[:, 1]

    poe = 1 - accuracy_score(y_pair, y_pred)
    auc = roc_auc_score(y_pair, y_proba)

    poe_row.append(poe)
    auc_row.append(auc)

# Generate and print results table
table = pd.DataFrame([poe_row, auc_row],
                     index=['PoE', 'AUC'],
                     columns=[f'{a} vs {b}' for a, b in pairs])

print("Evaluation Table on 256x256 - attack vs original:")
print(table.to_string(float_format="%.4f"))



Evaluation Table on 256x256 - attack vs original:
     mf3_90 vs orig  mf5_90 vs orig
PoE          0.5418          0.5269
AUC          0.4513          0.4196


# table-3

In [19]:
##############################################################################################
#  512 x 384 mf3_90 mf5_90 avg gau res jpeg ###############################################################################################

import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score
import pandas as pd

# Load features with labels
def load_data_with_labels(file_path):
    data = np.load(file_path)
    X = data[:, :-1]
    y = data[:, -1]
    return X, y

# Load 256x256 datasets
X_orig_512, y_orig_512 = load_data_with_labels('512x384_orig_unc.npy')
X_avg_512, y_avg_512 = load_data_with_labels('512x384_avg_unc.npy')
X_gau_512, y_gau_512 = load_data_with_labels('512x384_gau_unc.npy')
X_jpeg_512, y_jpeg_512 = load_data_with_labels('512x384_jpeg_90.npy')
X_res_512, y_res_512 = load_data_with_labels('512x384_res_unc.npy')
X_mf3_512, y_mf3_512 = load_data_with_labels('512x384_mf3_90.npy')
X_mf5_512, y_mf5_512 = load_data_with_labels('512x384_mf5_90.npy')

# Use previously defined `test_indices` (from 512x384_orig_unc)
# If running this separately, you need to re-import or re-calculate them
# For context:
# indices = np.arange(len(y_orig_512))
# train_indices, test_indices = train_test_split(indices, test_size=0.25, random_state=42, stratify=y_orig_512)

# Replace with appropriate length if you don't have it already
# For example purposes, using this (but use your actual test_indices!):
test_indices = np.load('test_indices_1.npy')  # load from file if previously saved

# Define dataset dictionary for 256x256
data_dict_512 = {
    'avg': (X_avg_512, y_avg_512),
    'gau': (X_gau_512, y_gau_512),
    'jpeg': (X_jpeg_512, y_jpeg_512),
    'res': (X_res_512, y_res_512),
    'mf3_90': (X_mf3_512, y_mf3_512),
    'mf5_90': (X_mf5_512, y_mf5_512)
}

# Define the same comparison pairs
pairs = [
    ('mf3_90', 'avg'), ('mf3_90', 'gau'), ('mf3_90', 'jpeg'), ('mf3_90', 'res'),
    ('mf5_90', 'avg'), ('mf5_90', 'gau'), ('mf5_90', 'jpeg'), ('mf5_90', 'res')
]

# Evaluate using the model previously trained on 512x384
poe_row = []
auc_row = []

for a, b in pairs:
    X_a, y_a = data_dict_512[a]
    X_b, y_b = data_dict_512[b]

    Xa_test = X_a[test_indices]
    Xb_test = X_b[test_indices]

    ya_test = np.ones(len(test_indices))  # attack A
    yb_test = np.zeros(len(test_indices))  # attack B

    X_pair = np.vstack([Xa_test, Xb_test])
    y_pair = np.hstack([ya_test, yb_test])

    y_pred = model_1.predict(X_pair)
    y_proba = model_1.predict_proba(X_pair)[:, 1]

    poe = 1 - accuracy_score(y_pair, y_pred)
    auc = roc_auc_score(y_pair, y_proba)

    poe_row.append(poe)
    auc_row.append(auc)

# Generate evaluation table
table = pd.DataFrame([poe_row, auc_row],
                     index=['PoE', 'AUC'],
                     columns=[f'{a} vs {b}' for a, b in pairs])

print("Evaluation Table on 512x384 data:")
print(table.to_string(float_format="%.4f"))


Evaluation Table on 512x384 data:
     mf3_90 vs avg  mf3_90 vs gau  mf3_90 vs jpeg  mf3_90 vs res  mf5_90 vs avg  mf5_90 vs gau  mf5_90 vs jpeg  mf5_90 vs res
PoE         0.2776         0.2776          0.2791         0.2791         0.0642         0.0642          0.0657         0.0657
AUC         0.9579         0.9908          0.9647         0.9883         0.9848         0.9929          0.9854         0.9898


In [20]:
##############################################################################################
#  256 x 256 mf3_90 mf5_90 avg gau res jpeg ###############################################################################################

import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score
import pandas as pd

# Load features with labels
def load_data_with_labels(file_path):
    data = np.load(file_path)
    X = data[:, :-1]
    y = data[:, -1]
    return X, y

# Load 256x256 datasets
X_orig_256, y_orig_256 = load_data_with_labels('256x256_orig_unc.npy')
X_avg_256, y_avg_256 = load_data_with_labels('256x256_avg_unc.npy')
X_gau_256, y_gau_256 = load_data_with_labels('256x256_gau_unc.npy')
X_jpeg_256, y_jpeg_256 = load_data_with_labels('256x256_jpeg_90.npy')
X_res_256, y_res_256 = load_data_with_labels('256x256_res_unc.npy')
X_mf3_256, y_mf3_256 = load_data_with_labels('256x256_mf3_90.npy')
X_mf5_256, y_mf5_256 = load_data_with_labels('256x256_mf5_90.npy')

# Use previously defined `test_indices` (from 512x384_orig_unc)
# If running this separately, you need to re-import or re-calculate them
# For context:
# indices = np.arange(len(y_orig_512))
# train_indices, test_indices = train_test_split(indices, test_size=0.25, random_state=42, stratify=y_orig_512)

# Replace with appropriate length if you don't have it already
# For example purposes, using this (but use your actual test_indices!):
test_indices = np.load('test_indices_1.npy')  # load from file if previously saved

# Define dataset dictionary for 256x256
data_dict_256 = {
    'avg': (X_avg_256, y_avg_256),
    'gau': (X_gau_256, y_gau_256),
    'jpeg': (X_jpeg_256, y_jpeg_256),
    'res': (X_res_256, y_res_256),
    'mf3_90': (X_mf3_256, y_mf3_256),
    'mf5_90': (X_mf5_256, y_mf5_256)
}

# Define the same comparison pairs
pairs = [
    ('mf3_90', 'avg'), ('mf3_90', 'gau'), ('mf3_90', 'jpeg'), ('mf3_90', 'res'),
    ('mf5_90', 'avg'), ('mf5_90', 'gau'), ('mf5_90', 'jpeg'), ('mf5_90', 'res')
]

# Evaluate using the model previously trained on 512x384
poe_row = []
auc_row = []

for a, b in pairs:
    X_a, y_a = data_dict_256[a]
    X_b, y_b = data_dict_256[b]

    Xa_test = X_a[test_indices]
    Xb_test = X_b[test_indices]

    ya_test = np.ones(len(test_indices))  # attack A
    yb_test = np.zeros(len(test_indices))  # attack B

    X_pair = np.vstack([Xa_test, Xb_test])
    y_pair = np.hstack([ya_test, yb_test])

    y_pred = model_1.predict(X_pair)
    y_proba = model_1.predict_proba(X_pair)[:, 1]

    poe = 1 - accuracy_score(y_pair, y_pred)
    auc = roc_auc_score(y_pair, y_proba)

    poe_row.append(poe)
    auc_row.append(auc)

# Generate evaluation table
table = pd.DataFrame([poe_row, auc_row],
                     index=['PoE', 'AUC'],
                     columns=[f'{a} vs {b}' for a, b in pairs])

print("Evaluation Table on 256x256 data:")
print(table.to_string(float_format="%.4f"))


Evaluation Table on 256x256 data:
     mf3_90 vs avg  mf3_90 vs gau  mf3_90 vs jpeg  mf3_90 vs res  mf5_90 vs avg  mf5_90 vs gau  mf5_90 vs jpeg  mf5_90 vs res
PoE         0.5015         0.5060          0.4910         0.5164         0.4866         0.4910          0.4761         0.5015
AUC         0.6516         0.6179          0.6557         0.4902         0.6056         0.5743          0.6087         0.4542


In [21]:
###################################################################################################
# 512 x384 mf35_90 all
#####################################################################################################

import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score
import pandas as pd
import joblib  # optional if you decide to save/load model

# Define function again (in case you're running standalone)
def load_data_with_labels(file_path):
    data = np.load(file_path)
    X = data[:, :-1]
    y = data[:, -1]
    return X, y

# Load previously used test indices
# X_all_512, y_all_512 = load_data_with_labels('512x384_all_unc.npy')
# indices = np.arange(len(y_all_512))
# _, test_indices = train_test_split(indices, test_size=0.25, random_state=42, stratify=y_all_512)

test_indices = np.load('test_indices_2.npy')

# Load trained model (if saved to file, otherwise assume `model` is in memory)
# model = joblib.load('trained_model_512_poly.pkl')

# Load 256x256 datasets
X_all_512, y_all_512 = load_data_with_labels('512x384_all_unc.npy')
X_mf35_512, y_mf35_512 = load_data_with_labels('512x384_mf35_90.npy')

# Create dataset dictionary
data_dict = {
    '512_all': (X_all_512, y_all_512),
    '512_mf35_90': (X_mf35_512, y_mf35_512)
}

# Define comparison pair
pairs = [('512_mf35_90', '512_all')]

# Evaluate
poe_row = []
auc_row = []

for a, b in pairs:
    X_a, y_a = data_dict[a]
    X_b, y_b = data_dict[b]

    Xa_test = X_a[test_indices]
    Xb_test = X_b[test_indices]

    ya_test = np.ones(len(test_indices))  # label 1 for attack a
    yb_test = np.zeros(len(test_indices))  # label 0 for attack b

    X_pair = np.vstack([Xa_test, Xb_test])
    y_pair = np.hstack([ya_test, yb_test])

    y_pred = model_2.predict(X_pair)
    y_proba = model_2.predict_proba(X_pair)[:, 1]

    poe = 1 - accuracy_score(y_pair, y_pred)
    auc = roc_auc_score(y_pair, y_proba)

    poe_row.append(poe)
    auc_row.append(auc)

# Create result table
table = pd.DataFrame([poe_row, auc_row],
                     index=['PoE', 'AUC'],
                     columns=[f'{a} vs {b}' for a, b in pairs])

print("Evaluation Table (on 512x384 data):")
print(table.to_string(float_format="%.4f"))


 

Evaluation Table (on 512x384 data):
     512_mf35_90 vs 512_all
PoE                  0.0479
AUC                  0.9890


In [22]:
###################################################################################################
# 256 x 256 mf35_90 all
#####################################################################################################

import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score
import pandas as pd
import joblib  # optional if you decide to save/load model

# Define function again (in case you're running standalone)
def load_data_with_labels(file_path):
    data = np.load(file_path)
    X = data[:, :-1]
    y = data[:, -1]
    return X, y

# Load previously used test indices
# X_all_512, y_all_512 = load_data_with_labels('512x384_all_unc.npy')
# indices = np.arange(len(y_all_512))
# _, test_indices = train_test_split(indices, test_size=0.25, random_state=42, stratify=y_all_512)

test_indices = np.load('test_indices_2.npy')

# Load trained model (if saved to file, otherwise assume `model` is in memory)
# model = joblib.load('trained_model_512_poly.pkl')

# Load 256x256 datasets
X_all_256, y_all_256 = load_data_with_labels('256x256_all_unc.npy')
X_mf35_256, y_mf35_256 = load_data_with_labels('256x256_mf35_90.npy')

# Create dataset dictionary
data_dict = {
    '256_all': (X_all_256, y_all_256),
    '256_mf35_90': (X_mf35_256, y_mf35_256)
}

# Define comparison pair
pairs = [('256_mf35_90', '256_all')]

# Evaluate
poe_row = []
auc_row = []

for a, b in pairs:
    X_a, y_a = data_dict[a]
    X_b, y_b = data_dict[b]

    Xa_test = X_a[test_indices]
    Xb_test = X_b[test_indices]

    ya_test = np.ones(len(test_indices))  # label 1 for attack a
    yb_test = np.zeros(len(test_indices))  # label 0 for attack b

    X_pair = np.vstack([Xa_test, Xb_test])
    y_pair = np.hstack([ya_test, yb_test])

    y_pred = model_2.predict(X_pair)
    y_proba = model_2.predict_proba(X_pair)[:, 1]

    poe = 1 - accuracy_score(y_pair, y_pred)
    auc = roc_auc_score(y_pair, y_proba)

    poe_row.append(poe)
    auc_row.append(auc)

# Create result table
table = pd.DataFrame([poe_row, auc_row],
                     index=['PoE', 'AUC'],
                     columns=[f'{a} vs {b}' for a, b in pairs])

print("Evaluation Table (on 256x256 data):")
print(table.to_string(float_format="%.4f"))


 

Evaluation Table (on 256x256 data):
     256_mf35_90 vs 256_all
PoE                  0.4820
AUC                  0.5916


# table-4

In [23]:
#############################################################################################################
# 512 x 384 ori_90 mf3_90 mf5_90
#############################################################################################################


import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score
import pandas as pd

# Load features with labels
def load_data_with_labels(file_path):
    data = np.load(file_path)
    X = data[:, :-1]
    y = data[:, -1]
    return X, y

# Load specified 512x384 datasets
X_orig, y_orig = load_data_with_labels('512x384_jpeg_90.npy')

X_mf3_90, y_mf3_90 = load_data_with_labels('512x384_mf3_90.npy')
X_mf5_90, y_mf5_90 = load_data_with_labels('512x384_mf5_90.npy')

# Load saved test indices
test_indices = np.load('test_indices_1.npy')

# Define dataset dictionary
data_dict = {
    'orig_90': (X_orig, y_orig),

    'mf3_90': (X_mf3_90, y_mf3_90),
    'mf5_90': (X_mf5_90, y_mf5_90)
}

# Define comparison pairs
pairs = [
    ('mf3_90', 'orig_90'), ('mf5_90', 'orig_90')
]

# Evaluate using the previously trained SVM model (ensure `model_1` exists in memory)
poe_row = []
auc_row = []

for a, b in pairs:
    X_a, y_a = data_dict[a]
    X_b, y_b = data_dict[b]

    Xa_test = X_a[test_indices]
    Xb_test = X_b[test_indices]

    ya_test = np.ones(len(test_indices))  # label for attack
    yb_test = np.zeros(len(test_indices))  # label for original

    X_pair = np.vstack([Xa_test, Xb_test])
    y_pair = np.hstack([ya_test, yb_test])

    y_pred = model_1.predict(X_pair)
    y_proba = model_1.predict_proba(X_pair)[:, 1]

    poe = 1 - accuracy_score(y_pair, y_pred)
    auc = roc_auc_score(y_pair, y_proba)

    poe_row.append(poe)
    auc_row.append(auc)

# Generate and print results table
table = pd.DataFrame([poe_row, auc_row],
                     index=['PoE', 'AUC'],
                     columns=[f'{a} vs {b}' for a, b in pairs])

print("Evaluation Table on 512x384 - attack vs original:")
print(table.to_string(float_format="%.4f"))



Evaluation Table on 512x384 - attack vs original:
     mf3_90 vs orig_90  mf5_90 vs orig_90
PoE             0.2791             0.0657
AUC             0.9647             0.9854


In [24]:
#############################################################################################################
# 256 x 256 ori_90 mf3_90 mf5_90
#############################################################################################################


import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score
import pandas as pd

# Load features with labels
def load_data_with_labels(file_path):
    data = np.load(file_path)
    X = data[:, :-1]
    y = data[:, -1]
    return X, y

# Load specified 512x384 datasets
X_orig, y_orig = load_data_with_labels('256x256_jpeg_90.npy')
# X_mf3, y_mf3 = load_data_with_labels('256x256_mf3_unc.npy')
# X_mf5, y_mf5 = load_data_with_labels('256x256_mf5_unc.npy')
X_mf3_90, y_mf3_90 = load_data_with_labels('256x256_mf3_90.npy')
X_mf5_90, y_mf5_90 = load_data_with_labels('256x256_mf5_90.npy')

# Load saved test indices
test_indices = np.load('test_indices_1.npy')

# Define dataset dictionary
data_dict = {
    'orig_90': (X_orig, y_orig),
    # 'mf3': (X_mf3, y_mf3),
    # 'mf5': (X_mf5, y_mf5),
    'mf3_90': (X_mf3_90, y_mf3_90),
    'mf5_90': (X_mf5_90, y_mf5_90)
}

# Define comparison pairs
pairs = [
    ('mf3_90', 'orig_90'), ('mf5_90', 'orig_90')
]

# Evaluate using the previously trained SVM model (ensure `model_1` exists in memory)
poe_row = []
auc_row = []

for a, b in pairs:
    X_a, y_a = data_dict[a]
    X_b, y_b = data_dict[b]

    Xa_test = X_a[test_indices]
    Xb_test = X_b[test_indices]

    ya_test = np.ones(len(test_indices))  # label for attack
    yb_test = np.zeros(len(test_indices))  # label for original

    X_pair = np.vstack([Xa_test, Xb_test])
    y_pair = np.hstack([ya_test, yb_test])

    y_pred = model_1.predict(X_pair)
    y_proba = model_1.predict_proba(X_pair)[:, 1]

    poe = 1 - accuracy_score(y_pair, y_pred)
    auc = roc_auc_score(y_pair, y_proba)

    poe_row.append(poe)
    auc_row.append(auc)

# Generate and print results table
table = pd.DataFrame([poe_row, auc_row],
                     index=['PoE', 'AUC'],
                     columns=[f'{a} vs {b}' for a, b in pairs])

print("Evaluation Table on 256x256 - attack vs original:")
print(table.to_string(float_format="%.4f"))



Evaluation Table on 256x256 - attack vs original:
     mf3_90 vs orig_90  mf5_90 vs orig_90
PoE             0.4910             0.4761
AUC             0.6557             0.6087
